In [ ]:
# translation_images
import cv2
import numpy as np

img = cv2.imread('img/snoopy.jpg')

rows, cols = img.shape[:2]

# 변환 행렬, X축으로 10, Y축으로 20 이동
#M = np.float32([[1,0,10],[0,1,20]])
#이미지의 중심점을 기준으로 90도 회전 하면서 0.5배 Scale
M= cv2.getRotationMatrix2D((cols/2, rows/2),90, 0.5)

dst = cv2.warpAffine(img, M,(cols, rows))
cv2.imshow('Original', img)
#cv2.imshow('Translation', dst)
cv2.imshow('Rotation', dst)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# speeded-up_robust_features
import cv2
import numpy as np

img = cv2.imread('img/butterfly.jpg')
gray= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

surf = cv2.xfeatures2d.SURF_create(500)
kp, des = surf.detectAndCompute(gray, None)
print(len(kp))
img2 = cv2.drawKeypoints(gray,kp,None,(0,0,255),4)

cv2.imshow('img2', img2)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
# smoothing_images
'''
변환
좌표 x를 좌표 x’로 변환하는 함수
강체변환(Ridid-Body) : 크기 및 각도가 보존(ex; Translation, Rotation)
유사변환(Similarity) : 크기는 변하고 각도는 보존(ex; Scaling)
선형변환(Linear) : Vector 공간에서의 이동. 이동변환은 제외.
Affine : 선형변환과 이동변환까지 포함. 선의 수평성은 유지.(ex;사각형->평행사변형)
Perspective : Affine변환에 수평성도 유지되지 않음. 원근변환
'''
#-*- coding:utf-8 -*-
import cv2
import numpy as np

img = cv2.imread('img/snoopy.jpg')

# 행 : Height, 열:width
height, width = img.shape[:2]

'''
cv2.resize(img, dsize, fx, fy, interpolation)
Parameters:	
img – Image
dsize – Manual Size. 가로, 세로 형태의 tuple(ex; (100,200))
fx – 가로 사이즈의 배수. 2배로 크게하려면 2. 반으로 줄이려면 0.5
fy – 세로 사이즈의 배수
interpolation – 보간법
'''
# 이미지 축소
shrink = cv2.resize(img, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)

# Manual Size지정
zoom1 = cv2.resize(img, (width*2, height*2), interpolation=cv2.INTER_CUBIC)

# 배수 Size지정
zoom2 = cv2.resize(img, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)


cv2.imshow('Origianl', img)
cv2.imshow('Shrink', shrink)
cv2.imshow('Zoom1', zoom1)
cv2.imshow('Zoom2', zoom2)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# otsus_binarization
import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread('img/snoopy.jpg',0)

# global thresholding
ret1,th1 = cv2.threshold(img,127,255,cv2.THRESH_BINARY)

# Otsu's thresholding
ret2,th2 = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

# Otsu's thresholding after Gaussian filtering
blur = cv2.GaussianBlur(img,(5,5),0)
ret3,th3 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

# plot all the images and their histograms
images = [img, 0, th1,
          img, 0, th2,
          blur, 0, th3]
titles = ['Original Noisy Image','Histogram','Global Thresholding (v=127)',
          'Original Noisy Image','Histogram',"Otsu's Thresholding",
          'Gaussian filtered Image','Histogram',"Otsu's Thresholding"]

for i in range(3):
    plt.subplot(3,3,i*3+1),plt.imshow(images[i*3],'gray')
    plt.title(titles[i*3]), plt.xticks([]), plt.yticks([])
    plt.subplot(3,3,i*3+2),plt.hist(images[i*3].ravel(),256)
    plt.title(titles[i*3+1]), plt.xticks([]), plt.yticks([])
    plt.subplot(3,3,i*3+3),plt.imshow(images[i*3+2],'gray')
    plt.title(titles[i*3+2]), plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
# otsus_binarization2
import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread('img/snoopy.jpg',0)
blur = cv2.GaussianBlur(img,(5,5),0)

# find normalized_histogram, and its cumulative distribution function
hist = cv2.calcHist([blur],[0],None,[256],[0,256])
hist_norm = hist.ravel()/hist.max()
Q = hist_norm.cumsum()

bins = np.arange(256)

fn_min = np.inf
thresh = -1

for i in range(1,256):
    p1,p2 = np.hsplit(hist_norm,[i]) # probabilities
    q1,q2 = Q[i],Q[255]-Q[i] # cum sum of classes
    b1,b2 = np.hsplit(bins,[i]) # weights

    # finding means and variances
    m1,m2 = np.sum(p1*b1)/q1, np.sum(p2*b2)/q2
    v1,v2 = np.sum(((b1-m1)**2)*p1)/q1,np.sum(((b2-m2)**2)*p2)/q2

    # calculates the minimization function
    fn = v1*q1 + v2*q2
    if fn < fn_min:
        fn_min = fn
        thresh = i

# find otsu's threshold value with OpenCV function
ret, otsu = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
print (thresh,ret)

In [ ]:
# affine_transformation_images
# 좌표
'''
Affine Transformation은 선의 평행성은 유지가 되면서 이미지를 변환
이동, 확대, Scale, 반전까지 포함된 변환 
Affine 변환을 위해서는 3개의 Match가 되는 점이 있으면 변환행렬
'''
import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread('img/snoopy.jpg')
rows, cols, ch = img.shape

pts1 = np.float32([[200,100],[400,100],[200,200]])
pts2 = np.float32([[200,300],[400,200],[200,400]])

# pts1의 좌표에 표시. Affine 변환 후 이동 점 확인.
cv2.circle(img, (200,100), 10, (255,0,0),-1)
cv2.circle(img, (400,100), 10, (0,255,0),-1)
cv2.circle(img, (200,200), 10, (0,0,255),-1)

M = cv2.getAffineTransform(pts1, pts2)

dst = cv2.warpAffine(img, M, (cols,rows))

plt.subplot(121),plt.imshow(img),plt.title('image')
plt.subplot(122),plt.imshow(dst),plt.title('Affine')
plt.show()

In [ ]:
# binary_robust_independent_elementary_features
import numpy as np
import cv2
from matplotlib import pyplot as plt

img = cv2.imread('img/butterfly.jpg', 0)
star = cv2.xfeatures2d.StarDetector_create() # STAR 검출기 초기화
brief = cv2.xfeatures2d.BriefDescriptorExtractor_create() 

kp = star.detect(img,None) # 특징점 검출
kp, des = brief.compute(img, kp) # 특징점의 기술자(Descriptor)를 계산
img2 = cv2.drawKeypoints(img, kp, None, (255,0,0))

cv2.imshow('img2', img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# bitwise_operations
import cv2
import numpy as np

img1 = cv2.imread('img/logo.jpg')
img2 = cv2.imread('img/test.jpg')

# 삽입할 이미지의 row, col, channel정보
rows, cols, channels = img1.shape

# 대상 이미지에서 삽입할 이미지의 영역을 추출
roi = img2[0:rows, 0:cols]

#mask를 만들기 위해서 img1을 gray로 변경후 binary image로 전환
#mask는 logo부분이 흰색(255), 바탕은 검은색(0)
#mask_inv는 logo부분이 검은색(0), 바탕은 흰색(255)

img2gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
ret, mask = cv2.threshold(img2gray, 10, 255, cv2.THRESH_BINARY)
mask_inv = cv2.bitwise_not(mask)

#bitwise_and 연산자는 둘다 0이 아닌 경우만 값을 통과 시킴.
#즉 mask가 검정색이 아닌 경우만 통과가 되기때문에 mask영역 이외는 모두 제거됨.
#아래 img1_fg의 경우는 bg가 제거 되고 fg(logo부분)만 남게 됨.
#img2_bg는 roi영역에서 logo부분이 제거되고 bg만 남게 됨.
img1_fg = cv2.bitwise_and(img1, img1, mask=mask)
img2_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)

#2개의 이미지를 합치면 바탕은 제거되고 logo부분만 합쳐짐.
dst = cv2.add(img1_fg, img2_bg)

#합쳐진 이미지를 원본 이미지에 추가.
img2[0:rows, 0:cols] = dst

cv2.imshow('res', img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# grabCut_algorithm
'''
GrabCut 알고리즘은 이미지에서 배경이 아닌 전경에 해당하는 이미지를 추출
이미지에서 한번에 전경을 추출해 내는 것이 아닌 사용자와의 상호 작용을 통해 단계적으로 전경을 추출
이 상호작용은 크게 2가지 단계로 진행되는데, 
첫번째는 이미지에서 전경이 포함되는 영역을 사각형으로 대략적으로 지정 
단, 이때 지정한 사각형 영역 안에는 전경이 모두 포함
그리고 두번째는 첫번째에서 얻어진 전경 이미지의 내용중 포함되어진 배경 부분은 어디인지, 
누락된 전경 부분은 어디인지를 마킹하면 이를 이용해 다시 전경 이미지가 새롭게 추출
'''
import numpy as np
import cv2
from matplotlib import pyplot as plt

img = cv2.imread('img/test.jpg')
mask = np.zeros(img.shape[:2],np.uint8)
# 전경에 해당하는 화소 위치에 값이 설정되는데, 0 또는 2 값은 배경이고 1 또는 3인 전경

bgdModel = np.zeros((1,65),np.float64)
fgdModel = np.zeros((1,65),np.float64)

rect = (50,50,450,290)
cv2.grabCut(img,mask,rect,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_RECT)

mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
img = img*mask2[:,:,np.newaxis]
plt.imshow(img),plt.colorbar(),plt.show()

In [ ]:
# grabCut_algorithm2
import numpy as np
import cv2
from matplotlib import pyplot as plt
img = cv2.imread('img/test.jpg')
mask = np.zeros(img.shape[:2],np.uint8)
bgdModel = np.zeros((1,65),np.float64)
fgdModel = np.zeros((1,65),np.float64)
# Step 1
rect = (50,50,450,290)
cv2.grabCut(img,mask,rect,bgdModel,fgdModel,1,cv2.GC_INIT_WITH_RECT)
# Step 2
newmask = cv2.imread('img/newmask2.png',0)

mask[newmask == 0] = 0
mask[newmask == 255] = 1
cv2.grabCut(img,mask,None,bgdModel,fgdModel,1,cv2.GC_INIT_WITH_MASK)
mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
img = img*mask2[:,:,np.newaxis]
plt.imshow(img),plt.colorbar(),plt.show()

In [ ]:
# measuring_performance
'''
cv2.getTickCount function returns the number of clock-cycles after a reference event 
(like the moment machine was switched ON) to the moment this function is called. 
So if you call it before and after the function execution, 
you get number of clock-cycles used to execute a function.

cv2.getTickFrequency function returns the frequency of clock-cycles, 
or the number of clock-cycles per second.
'''
import cv2
import numpy as np

img1 = cv2.imread('img/test.jpg')

x = range(5,49,2)

e1 = cv2.getTickCount()
for i in x:
    img1 = cv2.medianBlur(img1,i)
e2 = cv2.getTickCount()
t = (e2 - e1)/cv2.getTickFrequency()
print(t)